# Consolation Prize

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from utils.trainFold import getVector

In [3]:
import sys
sys.path.append('../dsbase/src/main')
from AdaBoostClassificationDSBase import AdaBoostClassificationDSBaseModel

## Loading the test dataset

In [4]:
# cat datasets/test.csv | head -n +1001 > datasets/test_reduced.csv

In [5]:
df = pd.read_csv('datasets/test_reduced.csv')

In [6]:
# TODO Clean as training

## Defining the Fold X processing 

### Defining helping functions 

In [7]:
def loadColumnCategoricalOrder(df, columns_categorical):
    columns_categorical_order_dict = {}
    for x in columns_categorical:
        columns_categorical_order_dict[x] = np.where(df.columns == x)[0][0]
    return columns_categorical_order_dict

In [8]:
def loadColumnCategoricalVectors(fold_id, columns_categorical):
    columns_categorical_vectors_dict = {}
    out_path = 'models/fold' + str(fold_id)
    for c in columns_categorical:
        vec = np.load('models/fold' + str(fold_id) + "/" + c + ".sav.npy")
        columns_categorical_vectors_dict[c] = vec
    return columns_categorical_vectors_dict

In [9]:
def loadModel(fold_id):
   # --------------------------------------
    # Load the i-th model and process
    print('   loading model ...')    
    model = AdaBoostClassificationDSBaseModel('AB2',None,None,None,None,None,None)
    model.load('models/fold' + str(fold_id))
    return model

In [19]:
def calculate(x, cc, cc_o, cc_v, model, df_clean):    
    xp = x.values
    mi=xp[0]
    xnp = xp[1:]
    acc=0
    try:
        #print('mi: ' + mi)
        df_clean.loc[mi]
        print(' process:')
        for c in cc:
            index = cc_o[c] + acc
            vec = cc_v[c]
            new = getVector(xnp[index], vec)
            print('  new: ' + str(new))
            xnp = np.delete(xnp, index)
            print('  xnp: ' + str(xnp))
            xnp = np.insert(xnp, index, new)
            print('  xnp: ' + str(xnp))
            acc += (new.size - 1)
        pre_result = model.scalerX.transform(xnp.reshape(1,-1))
        result = model.model.predict_proba(pre_result)
        return result[0,1]
    except KeyError:
        return 0.5

### calculating support variables

In [11]:
df_w = df.drop(['Census_ProcessorClass','Census_InternalBatteryType','DefaultBrowsersIdentifier','Census_IsFlightingInternal','Census_ThresholdOptIn','Census_IsWIMBootEnabled','PuaMode'], axis=1)
columns_categorical = df_w.select_dtypes(include=['object']).columns[1:]

In [12]:
df_w.dropna(inplace=True)

In [13]:
df_w.set_index(['MachineIdentifier'], inplace=True)

In [14]:
df_w.shape

(972, 74)

In [15]:
cc_order = loadColumnCategoricalOrder(df_w,columns_categorical)

In [20]:
i = 2
print('-------- Process Fold ',i,' -------------------')
print('loading vectors ...')
cc_values_f = loadColumnCategoricalVectors(i,columns_categorical)
print('loading model ...')
model_f = loadModel(i)
print('applying folding prediction ...')
df['HasDetections'] = df.apply(func=calculate, axis=1, args=(columns_categorical, cc_order, cc_values_f, model_f, df_w))

-------- Process Fold  2  -------------------
loading vectors ...
loading model ...
   loading model ...
initiating empty model AB2. AdaBoostClassification
loading model: models/fold2/AdaBoostClassification_AB2.sav
applying folding prediction ...
mi: 0000010489e3af074adeac69c53e555e
mi: 00000176ac758d54827acd545b6315a5
mi: 0000019dcefc128c2d4387c1273dae1d
mi: 0000055553dc51b1295785415f1a224d
mi: 00000574cefffeca83ec8adf9285b2bf
mi: 000007ffedd31948f08e6c16da31f6d1
mi: 000008f31610018d898e5f315cdf1bd1
mi: 00000a3c447250626dbcc628c9cbc460
mi: 00000b6bf217ec9aef0f68d5c6705897
mi: 00000b8d3776b13e93ad83676a28e4aa
mi: 00000dec341e29f26b92c3be03640bdc
mi: 00000e658ce75c1e2a3bb47bcc3b08f3
mi: 0000102ff65968bbdc04b69073434b05
mi: 000011236a5dc4ff119541c42bb4287e
mi: 0000124d8811c1a5b5848c4d730cfbf8
mi: 000015a88b949c982bc80ec8e32b726d
mi: 0000187a73c37cab54bd5be81c16c71f
mi: 00001be80a499e95d5e68151b6470aa2
mi: 00001dcfc3f82d68d6eae9cad4a3e07c
mi: 00001ec5ccb1deb10e7f9facb5738bbb
mi: 00001fb4b

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


  xnp: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 7.0 0 nan 41571.0 2.0 1.0 1 203 143782.0 18.0 255.0 0 0 0 'x64' '10.0.0.0'
 16299 768 'rs3' '16299.637.amd64fre.rs3_release_svc.180808-1748' 'Home'
 1.0 0 nan 0.0 117.0 'RequireAdmin' 1.0 1.0 'Desktop' 'Windows.Desktop'
 516.0 188345.0 4.0 5.0 2344.0 nan 114473.0 'SSD' 113165.0 0 8192.0
 'Desktop' 23.8 1920.0 1080.0 'Desktop' nan 4294967295.0 '10.0.16299.666'
 'amd64' 'rs3_release' 16299 666 'Core' 'CORE' 'Upgrade' 39.0 160 'UNKNOWN'
 0 'IS_GENUINE' 'Retail' nan 0.0 'Retail' nan 142.0 9849.0 0 nan 0.0 1 1
 0.0 0.0 7.0]
  xnp: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0

ValueError: ('cannot label index with a null key', 'occurred at index 32')

## Save final dataset 

In [ ]:
df[df['HasDetections']!=0.5]

In [ ]:
df['MachineIdentifier','HasDetections'].to_csv('datasets/submission.csv')

# End of submission setting!! 